In [1]:
!pip install torch transformers scikit-learn

In [5]:
import torch

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

# Replace with your actual Hugging Face repository name
model_name = "srikar-v05/ner-bert"

# Load the model and tokenizer from Hugging Face Hub
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Move the model to the appropriate device
model.to(device)

# Set the model to evaluation mode
model.eval()



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
df = pd.read_csv('10000_Images_Samples.csv')
# Encode entity_value to numerical labels
label_encoder = LabelEncoder()
df['encoded_entity_value'] = label_encoder.fit_transform(df['entity_value'])

In [12]:
import torch

def predict(extracted_text, entity_name):
    input_text = f"{extracted_text} [SEP] entity_name: {entity_name}"
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Move inputs to the appropriate device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Get the predicted class
    predicted_class = torch.argmax(outputs.logits, dim=1).cpu().item()

    # Assuming you have a label encoder to get the original entity value
    predicted_value = label_encoder.inverse_transform([predicted_class])[0]

    return predicted_value

In [18]:
# Install PaddlePaddle (with GPU support)
!pip install paddlepaddle-gpu==2.5.0.post117 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

# Install PaddleOCR
!pip install paddleocr

# Install additional required libraries
!pip install pillow requests

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.7/546.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [20]:
from paddleocr import PaddleOCR
import requests
from PIL import Image
from io import BytesIO
import numpy as np

/usr/local/lib/python3.10/dist-packages/paddle/fluid/framework.py:549: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(


In [21]:
# Initialize PaddleOCR once
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Set language to English

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:17<00:00, 234kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:18<00:00, 559kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 139kiB/s]

[2024/09/21 10:07:51] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [22]:
def extract_text_from_image(image_url):
    """
    Extracts text from an image using PaddleOCR with error handling.

    Args:
        image_url (str): URL of the image to perform OCR on.

    Returns:
        str: Full extracted text from the image, or an error message.
    """
    try:
        # Load the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Check if the request was successful
        img = Image.open(BytesIO(response.content))

        # Convert the image to a numpy array
        img_np = np.array(img)

        # Perform OCR on the image
        result = ocr.ocr(img_np)

        # Check if result is empty or None
        if result and result[0]:
            # Extract text from the result
            extracted_text = ' '.join([line[-1][0] for line in result[0]])
            return extracted_text
        else:
            return "No text found in image."

    except requests.exceptions.RequestException as e:
        return f"Error loading image: {str(e)}"
    except TypeError as e:
        return f"Type error during processing: {str(e)}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [29]:
# Example usage
extracted_text = extract_text_from_image(input("Enter the URL: "))
entity_name = input("Enter the name of the entity: ")
predicted_value = predict(extracted_text, entity_name)
print(f"Predicted Entity Value: {predicted_value}")


Enter the URL: https://m.media-amazon.com/images/I/71u9qpY7TtL.jpg
[2024/09/21 10:12:59] ppocr DEBUG: dt_boxes num : 8, elapsed : 1.6476101875305176
[2024/09/21 10:13:00] ppocr DEBUG: cls num  : 8, elapsed : 0.15911149978637695
[2024/09/21 10:13:01] ppocr DEBUG: rec_res num  : 8, elapsed : 1.650170087814331
Enter the name of the entity: item_weight
Predicted Entity Value: 2.0 gram
